# 2_Extract

In this notebook I will create new variables and check for redundant variables that i can exclude 

# Import libraries
Import the necessary libraries that we are going to use further on.

In [1]:
import pandas as pd
import numpy as np
import function_utils as futils
pd.set_option('display.float_format', '{:.2f}'.format) # disable scientific notation
pd.set_option('display.max_rows', None) # display all rows

# Import Dataset
Import the  dataset

In [2]:
# Read the pickle file 'df_preprocessed.pkl' and store the data in a DataFrame called df
df = pd.read_pickle('data_preprocessed.pkl')

# Create a copy of the df DataFrame and assign it to preprocessed_df
preprocessed_df = df.copy()

# Display the first 5 rows of the case_study_df DataFrame
display(preprocessed_df.head(5))

,vehicle_ID,selling_country,buyer_country,make,model,mileage,body_type,fuel_type,colour,registration_date,...,b2c_price,stock_turnover,selling_year,selling_month,selling_day,registration_year,registration_month,registration_day,deviation_b2B,deviation_b2C
0,V000001,IT,IT,Toyota,4Runner,138366,SUV,Electric,Blue,2023-12-16,...,15797.47,26,2024,12,15,2023,12,16,0.15,-0.06
1,V000002,PT,PT,Chevrolet,Equinox,111565,SUV,Diesel,Silver,2020-05-11,...,10743.26,53,2024,5,10,2020,5,11,0.12,-0.12
2,V000003,ES,ES,Chevrolet,Equinox,139601,SUV,Electric,Black,2023-03-21,...,16305.35,46,2024,3,20,2023,3,21,0.12,-0.13
3,V000004,FR,FR,Porsche,Boxster,144300,Convertible,CNG,White,2015-10-02,...,18029.74,98,2024,9,29,2015,10,2,-0.10,-0.14
4,V000005,DE,DE,Subaru,Impreza,144016,Sedan,CNG,Grey,2014-03-15,...,5539.80,50,2024,3,12,2014,3,15,0.03,-0.16


In [3]:
# Check if data types were preserved
preprocessed_df.dtypes

vehicle_ID                    object
selling_country               object
buyer_country                 object
make                          object
model                         object
mileage                        int64
body_type                     object
fuel_type                     object
colour                        object
registration_date     datetime64[ns]
selling_date          datetime64[ns]
selling_price                float64
condition_grade               object
conversion_rate              float64
b2b_price                    float64
b2c_price                    float64
stock_turnover                 int64
selling_year                   int64
selling_month                  int64
selling_day                    int64
registration_year              int64
registration_month             int64
registration_day               int64
deviation_b2B                float64
deviation_b2C                float64
dtype: object

# 1. Feature Extraction
In this section, some new features will be calculated based on the variables we already have

## 1.1 Age of the vehicle at the time of sale

Calculate the age of the vehicle at the time of sale by subtracting the registration year from the sale year. Vehicles in the leasing segment are typically newer. Calculating age helps confirm whether vehicles from this vendor align with this profile.

In [4]:
# Function to calculate precise month difference
def calculate_months_diff(start_date, end_date):
    return (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) + \
           np.sign((end_date.day - start_date.day)) * 0.5  # Adjust for partial months

# Apply the function 
preprocessed_df['vehicle_age_sale_time_months'] = preprocessed_df.apply(lambda x: calculate_months_diff(x['registration_date'], x['selling_date']), axis=1)


In [5]:
# Check the new variable summary stats
preprocessed_df['vehicle_age_sale_time_months'].describe() 

count   10002.00
mean       66.13
std        34.42
min        11.50
25%        35.50
50%        71.50
75%        95.50
max       179.50
Name: vehicle_age_sale_time_months, dtype: float64

## 1.2 Price Deviation (B2B and B2C)

Already calculated in 1_Preprocess notebook

Evaluates how selling prices compare to market benchmarks. High positive deviations suggest strong pricing performance. Comparing deviations for the leasing segment vs. other segments highlights opportunities or risks in adopting the new vendor.

## 1.3  Price to mileage ratio

A high price-to-mileage ratio often indicates vehicles that retain value well. Leasing vehicles generally perform better here due to lower mileage.

In [6]:
preprocessed_df['price_to_mileage_ratio'] = preprocessed_df['selling_price'] / preprocessed_df['mileage']


## 1.4 Leasing Segment

Classify vehicles into leasing or non-leasing segments based on Age, Mileage and Model

### 1.4.1 Age

Here i will check the distribution of the vehicle's age by decile to see in which value i will set the treshold for considering a leasing car in the leasing segment

In [7]:
_, age_aggregated = futils.create_decile_segment(
    df=preprocessed_df,
    sort_column='vehicle_age_sale_time_months',
    segment_prefix='decile',
    n_deciles=10,
    ascending=True,
    agg_columns={'vehicle_age_sale_time_months': ['min', 'max', 'mean']}
)

In [8]:
age_aggregated

,decile_vehicle_age_sale_time_months,vehicle_age_sale_time_months_min,vehicle_age_sale_time_months_max,vehicle_age_sale_time_months_mean
0,1,11.50,23.50,11.97
1,2,23.50,35.50,23.77
2,3,35.50,47.50,36.46
3,4,47.50,59.50,49.01
4,5,59.50,71.50,60.18
5,6,71.50,83.50,72.57
6,7,83.50,95.50,84.30
7,8,95.50,107.50,95.84
8,9,107.50,119.50,107.69
9,10,119.50,179.50,119.56


Age: Will consider leasing the ones with less than 36 months

### 1.4.2 Mileage

Here i will check the distribution of the vehicle's mileage by decile to see in which value i will set the treshold for considering a leasing car in the leasing segment

In [9]:
_, mileage_aggregated = futils.create_decile_segment(
    df=preprocessed_df,
    sort_column='mileage',
    segment_prefix='decile',
    n_deciles=10,
    ascending=True,
    agg_columns={'mileage': ['min', 'max', 'mean']}
)

In [10]:
mileage_aggregated

,decile_mileage,mileage_min,mileage_max,mileage_mean
0,1,5016,25078,15118.65
1,2,25080,43871,34587.33
2,3,43892,63375,53780.16
3,4,63380,83591,73590.96
4,5,83621,103241,93490.71
5,6,103242,122955,113347.34
6,7,122968,142447,133050.08
7,8,142448,162260,152519.12
8,9,162418,180570,171193.60
9,10,180591,999999,191131.82


Milleage: Will consider leasing the ones with less than 45k kilometers

### 1.4.3 Models

Here i could create a model (classification model) to see  which numeric variable (price, conversion_rate, etc) best describes the `model` and use that variable to make the same analysis as before. But for time purposes, i will just assume that price is a good variable to describe the `model`  

In [11]:
_, selling_price_aggregated = futils.create_decile_segment(
    df=preprocessed_df,
    sort_column='selling_price',
    segment_prefix='decile',
    n_deciles=10,
    ascending=True,
    agg_columns={'selling_price': ['min', 'max', 'mean']}
)

In [12]:
selling_price_aggregated

,decile_selling_price,selling_price_min,selling_price_max,selling_price_mean
0,1,1111.08,4392.35,3292.58
1,2,4393.29,6028.51,5246.74
2,3,6036.69,7539.68,6781.17
3,4,7540.37,9129.81,8307.48
4,5,9130.31,10841.08,9947.81
5,6,10841.43,12898.77,11840.78
6,7,12902.94,15420.59,14140.22
7,8,15420.92,19248.65,17185.21
8,9,19249.61,25762.99,22192.49
9,10,25775.99,87741.24,34957.01


Model: Will consider leasing the models with selling_price greater than 25k 

In [13]:
# Create 'leasing_segment' 
leasing_conditions = (
    (preprocessed_df['vehicle_age_sale_time_months'] < 36) &
    (preprocessed_df['mileage'] < 35000) &
    (preprocessed_df['selling_price'] > 25000)
)


preprocessed_df['leasing_segment'] = np.where(leasing_conditions, 'Leasing', 'Non-Leasing')


In [14]:
segment_counts = preprocessed_df['leasing_segment'].value_counts()
print("Segment Counts:\n", segment_counts)

Segment Counts:
 Non-Leasing    9811
Leasing         191
Name: leasing_segment, dtype: int64


#### For analysis purpose, i will now calculate new variables from the numeric ones creating intervals or categories that will help me build the presentation in a clear and easier way for non-technical people to  understand the graphs easier. For the choice of the intervals i will check the distribution by deciles and see which intervals are better

## 1.5 Mileage segment


In [15]:
# Define the bin edges
bins = [0, 5000, 7000, 10000,  np.inf]

# Define the labels for the segments
labels = ['<= 5.000 km', '5.001 - 7.000 km', '7.001 - 10.000 km', '>=10.001 km']

# Create a new column 'mileage_segment' based on the defined bins
preprocessed_df['mileage_segment'] = pd.cut(
    preprocessed_df['mileage'],
    bins=bins,
    labels=labels,
    right=True,  # Whether to include the right edge (default is True)
    include_lowest=True  # Whether the first interval should be left-inclusive
)

# Verify the segmentation
preprocessed_df['mileage_segment'].value_counts().sort_index()

<= 5.000 km             0
5.001 - 7.000 km      108
7.001 - 10.000 km     156
>=10.001 km          9738
Name: mileage_segment, dtype: int64

## 1.6 Selling price


In [16]:
# Define the bin edges
bins = [0, 15000, 25000, 35000, 50000, np.inf]  # 5 edges for 4 segments

labels = ['<= 15.000 €', '15.001 - 25.000 €', '25.001 - 35.000 €', '35.001 - 50.000 €','>=50.001 €']

#Create a new column 'selling_price_segment' based on the defined bins
preprocessed_df['selling_price_segment'] = pd.cut(
    preprocessed_df['selling_price'],
    bins=bins,
    labels=labels,
    right=True,          # Includes the right edge in the bin
    include_lowest=True  # Includes the lowest value in the first bin
)

# Verify the segmentation
preprocessed_df['selling_price_segment'].value_counts().sort_index()


<= 15.000 €          6839
15.001 - 25.000 €    2082
25.001 - 35.000 €     704
35.001 - 50.000 €     306
>=50.001 €             71
Name: selling_price_segment, dtype: int64

## 1.7 Conversion Rate

In [17]:
_, conversion_rate_aggregated = futils.create_decile_segment(
    df=preprocessed_df,
    sort_column='conversion_rate',
    segment_prefix='decile',
    n_deciles=10,
    ascending=True,
    agg_columns={'conversion_rate': ['min', 'max', 'mean']}
)

In [18]:
conversion_rate_aggregated

,decile_conversion_rate,conversion_rate_min,conversion_rate_max,conversion_rate_mean
0,1,55.05,76.30,70.20
1,2,76.30,79.39,77.89
2,3,79.39,82.10,80.79
3,4,82.10,84.20,83.16
4,5,84.20,86.23,85.17
5,6,86.23,88.34,87.27
6,7,88.34,90.52,89.42
7,8,90.52,92.84,91.68
8,9,92.84,95.84,94.12
9,10,95.87,100.00,98.71


In [19]:
# Define the bin edges
bins = [0, 82, 88, 92, np.inf]  # 5 edges for 4 segments

labels = ['<= 82.00%', '82.01 - 88.00 %', '88.01 - 92.00 %', '>=92.01 %']

# Create a new column 'selling_price_segment' based on the defined bins
preprocessed_df['conversion_rate_segment'] = pd.cut(
    preprocessed_df['conversion_rate'],
    bins=bins,
    labels=labels,
    right=True,          # Includes the right edge in the bin
    include_lowest=True  # Includes the lowest value in the first bin
)

# Verify the segmentation
segment_counts = preprocessed_df['conversion_rate_segment'].value_counts().sort_index()
print(segment_counts)

<= 82.00%          2961
82.01 - 88.00 %    2894
88.01 - 92.00 %    1779
>=92.01 %          2368
Name: conversion_rate_segment, dtype: int64


## 1.8 B2B Price

In [20]:
_, b2b_price_aggregated = futils.create_decile_segment(
    df=preprocessed_df,
    sort_column='b2b_price',
    segment_prefix='decile',
    n_deciles=10,
    ascending=True,
    agg_columns={'b2b_price': ['min', 'max', 'mean']}
)

In [21]:
b2b_price_aggregated

,decile_b2b_price,b2b_price_min,b2b_price_max,b2b_price_mean
0,1,1089.82,4092.63,3099.90
1,2,4098.05,5658.02,4921.46
2,3,5658.43,7068.90,6362.13
3,4,7072.52,8500.74,7785.82
4,5,8500.98,10152.42,9298.95
5,6,10152.46,12072.26,11076.43
6,7,12074.48,14434.79,13206.29
7,8,14437.12,17970.35,16082.03
8,9,17974.22,24304.71,20807.45
9,10,24323.04,92548.50,33723.81


In [22]:
# Define the bin edges
bins = [0, 15000, 25000, 35000, 50000, np.inf]  # 5 edges for 4 segments

labels = ['<= 15.000 €', '15.001 - 25.000 €', '25.001 - 35.000 €', '35.001 - 50.000 €','>=50.001 €']

# Create a new column 'selling_price_segment' based on the defined bins
preprocessed_df['b2b_price_segment'] = pd.cut(
    preprocessed_df['b2b_price'],
    bins=bins,
    labels=labels,
    right=True,          # Includes the right edge in the bin
    include_lowest=True  # Includes the lowest value in the first bin
)

# Verify the segmentation
segment_counts = preprocessed_df['b2b_price_segment'].value_counts().sort_index()
print(segment_counts)

<= 15.000 €          7192
15.001 - 25.000 €    1883
25.001 - 35.000 €     615
35.001 - 50.000 €     250
>=50.001 €             62
Name: b2b_price_segment, dtype: int64


## 1.9 B2C Price

In [23]:
_, b2c_price_aggregated = futils.create_decile_segment(
    df=preprocessed_df,
    sort_column='b2c_price',
    segment_prefix='decile',
    n_deciles=10,
    ascending=True,
    agg_columns={'b2c_price': ['min', 'max', 'mean']}
)

In [24]:
b2c_price_aggregated

,decile_b2c_price,b2c_price_min,b2c_price_max,b2c_price_mean
0,1,1000.00,4947.19,3728.93
1,2,4948.01,6817.89,5932.97
2,3,6819.47,8492.99,7663.82
3,4,8493.18,10310.64,9392.75
4,5,10312.68,12195.05,11227.08
5,6,12195.18,14547.02,13349.78
6,7,14548.95,17415.87,15918.61
7,8,17428.74,21670.69,19380.05
8,9,21678.74,29028.66,25000.54
9,10,29050.35,103033.82,39736.35


In [25]:
# Define the bin edges
bins = [0, 15000, 25000, 35000, 50000, np.inf]  # 5 edges for 4 segments

labels = ['<= 15.000 €', '15.001 - 25.000 €', '25.001 - 35.000 €', '35.001 - 50.000 €','>=50.001 €']

# Create a new column 'selling_price_segment' based on the defined bins
preprocessed_df['b2c_price_segment'] = pd.cut(
    preprocessed_df['b2c_price'],
    bins=bins,
    labels=labels,
    right=True,          # Includes the right edge in the bin
    include_lowest=True  # Includes the lowest value in the first bin
)

# Verify the segmentation
segment_counts = preprocessed_df['b2c_price_segment'].value_counts().sort_index()
print(segment_counts)

<= 15.000 €          6183
15.001 - 25.000 €    2347
25.001 - 35.000 €     904
35.001 - 50.000 €     424
>=50.001 €            144
Name: b2c_price_segment, dtype: int64


## 1.9 Stock Turnover

In [26]:
_, stock_turnover_aggregated = futils.create_decile_segment(
    df=preprocessed_df,
    sort_column='stock_turnover',
    segment_prefix='decile',
    n_deciles=10,
    ascending=True,
    agg_columns={'stock_turnover': ['min', 'max', 'mean']}
)

In [27]:
stock_turnover_aggregated

,decile_stock_turnover,stock_turnover_min,stock_turnover_max,stock_turnover_mean
0,1,15,28,22.74
1,2,28,35,31.85
2,3,35,39,37.16
3,4,39,44,41.59
4,5,44,48,45.47
5,6,48,52,49.63
6,7,52,57,54.02
7,8,57,62,59.35
8,9,62,73,66.58
9,10,73,156,93.03


In [28]:
# Define the bin edges
bins = [0, 40, 50, 60, np.inf]  # 5 edges for 4 segments

labels = ['<= 40 days', '41 - 50 days', '51 - 60 days', '>=61 days']

#Create a new column 'selling_price_segment' based on the defined bins
preprocessed_df['stock_turnover_segment'] = pd.cut(
    preprocessed_df['stock_turnover'],
    bins=bins,
    labels=labels,
    right=True,          # Includes the right edge in the bin
    include_lowest=True  # Includes the lowest value in the first bin
)

#Verify the segmentation
segment_counts = preprocessed_df['stock_turnover_segment'].value_counts().sort_index()
print(segment_counts)

<= 40 days      3228
41 - 50 days    2487
51 - 60 days    2001
>=61 days       2286
Name: stock_turnover_segment, dtype: int64


## Choosing variables for analysis

In [29]:
columns_to_drop = [
    'mileage',
    'vehicle_age_sale_time_months',
    'decile_stock_turnover',
    'decile_selling_price'
]

# Identify and drop existing columns
existing_columns_to_drop = [col for col in columns_to_drop if col in preprocessed_df.columns]
df_segmented = preprocessed_df.drop(columns=existing_columns_to_drop)


In [30]:
# Check  data types
df_segmented.dtypes

vehicle_ID                         object
selling_country                    object
buyer_country                      object
make                               object
model                              object
body_type                          object
fuel_type                          object
colour                             object
registration_date          datetime64[ns]
selling_date               datetime64[ns]
selling_price                     float64
condition_grade                    object
conversion_rate                   float64
b2b_price                         float64
b2c_price                         float64
stock_turnover                      int64
selling_year                        int64
selling_month                       int64
selling_day                         int64
registration_year                   int64
registration_month                  int64
registration_day                    int64
deviation_b2B                     float64
deviation_b2C                     

In [31]:
# Export dataset as pkl to preserve the data types
df_segmented.to_pickle('data_extract.pkl')